# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [11]:
order = pd.read_csv("Orders.csv")
order.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
Amount_customer = order.groupby(["CustomerID","Country"]).agg({"amount_spent":"sum"})
Amount_customer

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [24]:
bin_labels_5 = ['Normal', 'Preferred', 'Vip']
Amount_customer['Quantiles'] = pd.qcut(Amount_customer['amount_spent'],
                              q=[0, .75, .95,1],
                              labels=bin_labels_5)

In [30]:
# @Gonçalo, I was somehow not able to compute via strings from Quantiles. As I result I decided to create a helping column with a counter, which allows me to sum the result per Quantile
Amount_customer["Counter"]= 1
Amount_customer

,,amount_spent,Quantiles,Counter
CustomerID,Country,,,
12346,United Kingdom,77183.60,Vip,1
12347,Iceland,4310.00,Preferred,1
12348,Finland,1797.24,Preferred,1
12349,Italy,1757.55,Preferred,1
12350,Norway,334.40,Normal,1
...,...,...,...,...
18280,United Kingdom,180.60,Normal,1
18281,United Kingdom,80.82,Normal,1
18282,United Kingdom,178.05,Normal,1


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [50]:
Quantile_amount=Amount_customer.groupby(["Country","Quantiles"]).agg({'Counter':'sum'}).sort_values("Counter",ascending=False)
Quantile_amount

Counter
Country        Quantiles         
United Kingdom Normal        2987
               Preferred      756
               Vip            178
France         Normal          58
Germany        Normal          55
...                           ...
Brazil         Vip              0
               Preferred        0
RSA            Preferred        0
               Vip              0
Unspecified    Vip              0

[111 rows x 1 columns]

In [92]:
# I have converted colums into rows, as I was not able to only the VIP Customers from the previous df.
Customer_type_in_columns = Quantile_amount.pivot_table('Counter', ['Country'], 'Quantiles')
Customer_type_in_columns["Vip+Preferred"]= Customer_type_in_columns["Vip"] + Customer_type_in_columns["Preferred"]
Customer_type_in_columns = Customer_type_in_columns.reindex(['Vip', 'Preferred', 'Normal','Vip+Preferred'], axis=1)
Customer_type_in_columns

Quantiles,Vip,Preferred,Normal,Vip+Preferred
Country,,,,
Australia,1,3,5,4
Austria,0,1,10,1
Bahrain,0,0,2,0
Belgium,0,11,14,11
Brazil,0,0,1,0
Canada,0,1,3,1
Channel Islands,1,3,5,4
Cyprus,1,2,5,3
Czech Republic,0,0,1,0


In [100]:
Vip_customers = Customer_type_in_columns.iloc[:,0:1].sort_values("Vip",ascending=False).head(1)
Vip_customers

Quantiles,Vip
Country,
United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [112]:
VIP_Preferred = Customer_type_in_columns.iloc[:,:].sort_values("Vip+Preferred",ascending=False).head(1)
VIP_Preferred

# In general I was not able to select the columns. Maybe is not the expected way, but after spending long hours I cannot think of any other way.

Quantiles,Vip,Preferred,Normal,Vip+Preferred
Country,,,,
United Kingdom,178,756,2987,934
